# 进化算法的基本操作与实现
学习进化算法的笔记第一弹

## 参考资料
+ [基于DEAP库的Python进化算法从入门到入土--(一)进化算法的基本操作与实现](https://www.jianshu.com/p/8fa044ed9267)
+ deap源码与文档

## 遗传算法的优缺点
优点：

+ 泛用性强，对连续变量和离散变量都能适用；
+ 不需要导数信息，因此不要求适应度函数的连续和可微性质(或者说不需要问题内在机理的相关信息)；
+ 可以在解空间内大范围并行搜索；
+ 不容易陷入局部最优；
+ 高度并行化，并且容易与其他优化方法整合。

缺点：

+ 对于凸优化问题，相对基于梯度的优化方法（例如梯度下降法，牛顿/拟牛顿法）收敛速度更慢；
+ 进化算法需要在搜索空间投放大量个体来搜索最优解。对于高维问题，由于搜索空间随维度指数级膨胀，需要投放的个体数也大幅增长，会导致收敛速度变慢；
+ 设计编码方式、适应度函数以及变异规则需要大量经验。

## 问题定义、个体编码与创建初始族群
### 1.优化问题的定义：通过定义`base.Fitness`的子类来描述适应度这个指标
单目标优化：`creator.create('FitnessMin', base.Fitness, weights=(-1.0, ))`

在创建单目标优化问题时，weights用来指示最大化和最小化。此处-1.0即代表问题是一个最小化问题，对于最大化，应将weights改为正数，如1.0。

另外即使是单目标优化，weights也需要是一个tuple，以保证单目标和多目标优化时数据结构的统一。

对于单目标优化问题，weights 的绝对值没有意义，只要符号选择正确即可。

多目标优化：`creator.create('FitnessMulti', base.Fitness, weights=(-1.0, 1.0))`

对于多目标优化问题，weights用来指示多个优化目标之间的相对重要程度以及最大化最小化。如示例中给出的(-1.0, 1.0)代表对第一个目标函数取最小值，对第二个目标函数取最大值。

#### 补充内容： `creator.create(name, base, **kargs)` 

调用该函数后，会在creator模块中以base类为基类，设置kwargs中的属性而生成一个子类，该子类可以通过creator.name来访问。如果kargs传入的是类型，则会在实例化子类时自动调用该类型的无参构造器而给该实例创建一个成员变量；而当kargs传入对象时，则会将该对象作为这个子类的静态成员。

```python
create("Foo", list, bar=dict, spam=1)

# 等价于在creator模块中加入了以下定义

class Foo(list):
    spam = 1

    def __init__(self):
        self.bar = dict()
```

#### 补充内容：`base.Fitness`
用于比较结果好坏的一个指标。Fitness拥有value与weights两个属性，应当有相同的size。Fitness能够使用> < >= <= == != 等算符来进行比较value与weights两个向量的乘积（因此能够解释上文weights的设置原因），通过字典序进行判定。两个长度不同的Fitness能够进行比较，如果短的部分相同的话，长的那个更优。

### 2.个体编码
**实数编码(Value encoding)**：

直接用实数对变量进行编码。优点是不用解码，基因表达非常简洁，而且能对应连续区间。但是实数编码后搜索区间连续，因此容易陷入局部最优。

In [ ]:
"""
实数编码的DEAP实现。生成一个有五个染色体的个体
"""
from deap import base, creator, tools
import random
IND_SIZE = 5
creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) #优化目标：单变量，求最小值
creator.create('Individual', list, fitness = creator.FitnessMin) #创建Individual类，继承list

toolbox = base.Toolbox()
toolbox.register('Attr_float', random.random)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Attr_float, n=IND_SIZE)

ind1 = toolbox.Individual()
ind1

#### 补充内容：`toolbox.register(alias, function, *args, **kargs)`
在toolbox对象中注册一个函数alias，基于function并会自动填入args与kargs的参数。缺省参数在调用时提供。
```shell
>>> def func(a, b, c=3):
...     print(a, b, c)
...
>>> tools = Toolbox()
>>> tools.register("myFunc", func, 2, c=4)
>>> tools.myFunc(3)
2 3 4
```

#### 补充内容：`tools.initRepeat(container: function, func: function, n)`
实例化一个container对象，调用n次func，将结果置入container，然后返回container实例。

相当于 `return container(func(i) for in in range(n))`

练习：改为使用numpy生成相同的个体

In [ ]:
import numpy as np
IND_SIZE = 5
creator.create('IndividualWithNp', np.ndarray, fitness = creator.FitnessMin)

toolbox1 = base.Toolbox()
toolbox1.register('attr_float', np.random.rand)
toolbox1.register('get_individual', tools.initRepeat, creator.IndividualWithNp, toolbox1.attr_float, 5)
ind1 = toolbox1.get_individual()
ind1

#### 二进制编码(Binary encoding)：
在二进制编码中，用01两种数字模拟人类染色体中的4中碱基，用一定长度的01字符串来描述变量。其优点在于种群多样性大，但是需要解码，而且不连续，容易产生Hamming cliff（例如0111=7, 1000=8，改动了全部的4位数字之后，实际值只变动了1），在接近局部最优位置时，染色体稍有变动，就会使变量产生很大偏移（格雷编码（Gray coding）能够克服汉明距离的问题，但是实际问题复杂度较大时，格雷编码很难精确描述问题）。

**变量的二进制编码：**

由于通常情况下，搜索空间都是实数空间，因此在编码时，需要建立实数空间到二进制编码空间的映射。使用二进制不能对实数空间进行连续编码，但是可以在给定精度下对连续空间进行离散化。

以例子来说明如何对变量进行二进制编码，假设需要对一个在区间$[-2, 2]$上的变量进行二进制编码：

*选择编码长度*：在需要6位精度的情况下，我们需要将该区间离散为$(2+2)*10^6$个数。由于$2^{22}>4*10^6$，我们至少需要22位二进制数字来满足我们的精度要求。

*设置解码器*：将二进制数字$x^{bin}$转化为十进制$x^{dec}$之后（在python中可以用`int('Binary number', 2)`来实现），按照公式$x=-2+x^{dec}*4/(2^{22}-1)$，-1以得到一个在$[-2, 2]$区间内的实数。

*实现*：根据上面的实数编码不难发现，deap的个体概念与其所采用的底层数据存储无关（上面的示例与练习都分别采用了`list`与`np.ndarray`），所以各种个体编码本质上实现方式相同，均选择合适的数据结构进行存储即可。下面的示例作者依旧使用了list进行存储，没有契合二进制存储的特征（选择的数据结构应当支持`[]`运算符），有优化空间。这个示例主要是展示如何选择随机初始化函数。


In [ ]:
"""
二进制编码DEAP实现：

以随机生成一个长度为10的二进制编码为例，本身DEAP库中没有内置的Binary encoding，我们可以借助Scipy模块中的伯努利分布来生成一个二进制序列。
"""
from scipy.stats import bernoulli

creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) #优化目标：单变量，求最小值
creator.create('Individual', list, fitness = creator.FitnessMin) #创建Individual类，继承list

GENE_LENGTH = 10

toolbox = base.Toolbox()
toolbox.register('Binary', bernoulli.rvs, 0.5) #注册一个Binary的alias，指向scipy.stats中的bernoulli.rvs，概率为0.5
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Binary, n = GENE_LENGTH) #用tools.initRepeat生成长度为GENE_LENGTH的Individual

ind1 = toolbox.Individual()
ind1

#### 序列编码(Permutation encoding)：
通常在求解顺序问题时用到，例如TSP问题。序列编码中的每个染色体都是一个序列。
> 存在一个客观的序列，而染色体的值是查该序列的下标，整条染色体是这个原始序列的一个排列
> 下面的示例中，该序列就是`range(10)`

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("Indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("Individual", tools.initIterate, creator.Individual,toolbox.Indices)
ind1 = toolbox.Individual()
ind1

#### 补充内容：`np.random.permutation(x)`
重载函数。如果x是一个uint，那么会返回长度为x的一个乱序ndarray，等同于上面的例子中长度为10的染色体序列。而如果x是一个序列，则会直接打乱这个序列返回ndarray。

#### 补充内容：`initIterate: (container: function, generator: function)`
generator需要返回一个Iterable，initIterate相当于`return container(generator())`

#### 粒子(Particles)：
粒子是一种特殊个体，主要用于粒子群算法。相比普通的个体，它额外具有速度、速度限制并且能记录最优位置。

> TODO暂时跳过这部分，等之后学习到粒子群算法再返回

In [ ]:
"""
粒子的DEAP实现
"""
creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=None,
               smin=None, smax=None, best=None)

# 自定义的粒子初始化函数
def initParticle(pcls, size, pmin, pmax, smin, smax):
    part = pcls(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

toolbox = base.Toolbox()
toolbox.register("Particle", initParticle, creator.Particle, size=2, pmin=-6, pmax=6, smin=-3, smax=3) #为自己编写的initParticle函数注册一个alias "Particle"，调用时生成一个2维粒子，放在容器creator.Particle中，粒子的位置落在（-6,6）中，速度限制为（-3，3）

ind1 = toolbox.Particle()
print(ind1)
print(ind1.speed)
print(ind1.smin, ind1.smax)

# 结果：[-2.176528549934324, -3.0796558214905]
#[-2.9943676285620104, -0.3222138308543414]
#-3 3

print(ind1.fitness.valid)

# 结果：False
# 因为当前还没有计算适应度函数，所以粒子的最优适应度值还是invalid

### 3.初始族群的创建
#### 一般族群：
这是最常用的族群类型，族群中没有特别的顺序或者子族群。

一般族群的DEAP实现：`toolbox.register('population', tools.initRepeat, list, toolbox.individual)`

In [ ]:
"""
以二进制编码为例，以下代码可以生成由10个长度为5的随机二进制编码个体组成的一般族群
"""
# 定义问题
creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) # 单目标，最小化
creator.create('Individual', list, fitness = creator.FitnessMin)

# 生成个体
GENE_LENGTH = 5
toolbox = base.Toolbox() #实例化一个Toolbox
toolbox.register('Binary', bernoulli.rvs, 0.5)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Binary, n=GENE_LENGTH)

# 生成初始族群
N_POP = 10
toolbox.register('Population', tools.initRepeat, list, toolbox.Individual) # 这里注册时缺省了initRepeat函数的“n”参数，而在实际调用时提供。
toolbox.Population(n = N_POP)

#### 同类群(Demes)：
同类群即一个族群中包含几个子族群。在有些算法中，会使用本地选择(Local selection)挑选育种个体，这种情况下个体仅与同一邻域的个体相互作用。

In [ ]:
"""
同类群的DEAP实现：其实就是几个一般群的数组
"""
toolbox.register("deme", tools.initRepeat, list, toolbox.Individual) # 与一般群相同

DEME_SIZES = 10, 50, 100
population = [toolbox.deme(n=i) for i in DEME_SIZES] # 太长了不打印了

#### 粒子群(Swarm)：
粒子群中的所有粒子共享全局最优。在实现时需要额外传入全局最优位置与全局最优适应度给族群。

粒子群的DEAP实现：

```python
# 其实就是扩展了用于存储群的list，加入了全局最优位置和全局最优适应度两个属性。
creator.create("Swarm", list, gbest=None, gbestfit=creator.FitnessMax)
toolbox.register("swarm", tools.initRepeat, creator.Swarm, toolbox.particle)
```

在算法迭代时，需要更新该轮迭代中最优的位置和最优适应度。

## 评价
评价部分是根据任务的特性高度定制的，DEAP库中并没有预置的评价函数模版。

在使用DEAP时，需要注意的是，无论是单目标还是多目标优化，评价函数的返回值必须是一个tuple类型。

> 理由：评价函数的返回值将会被赋值给Fitness的values，而这个属性是一个tuple。

In [ ]:
# 定义问题
creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) #优化目标：单变量，求最小值
creator.create('Individual', list, fitness = creator.FitnessMin) #创建Individual类，继承list

# 生成个体
IND_SIZE = 5
toolbox = base.Toolbox()
toolbox.register('Attr_float', np.random.rand)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Attr_float, n=IND_SIZE)

# 生成初始族群
N_POP = 10
toolbox.register('Population', tools.initRepeat, list, toolbox.Individual)
pop = toolbox.Population(n = N_POP)

# 定义评价函数
def evaluate(individual):
  return sum(individual), #注意这个逗号，即使是单变量优化问题，也需要返回tuple

# 评价初始族群
toolbox.register('Evaluate', evaluate)
fitnesses = map(toolbox.Evaluate, pop)
for ind, fit in zip(pop, fitnesses):
  ind.fitness.values = fit
  print(ind.fitness.values)

## 配种选择
### 1.DEAP内置的选择操作
DEAP的tools模块中内置了13种选择操作，对全部选择算子的描述可以参考官方文档。
|函数| 解释
|---|---
|selTournament()|	锦标赛选择
|selRoulette()|	轮盘赌选择（不能用于最小化或者适应度会小于等于0的问题）
|selNSGA2()|	NSGA-II选择，适用于多目标遗传算法
|selSPEA2()|	SPEA2选择，目前版本(ver 1.2.2)的该函数实现有误，没有为个体分配距离，不建议使用。
|selRandom()|	有放回的随机选择
|selBest()|	选择最佳
|selWorst()|	选择最差
|selTournamentDCD()|	Dominance/Crowding distance锦标赛选择，目前版本的实现也有些问题
|selDoubleTournament()|	Size+Fitness双锦标赛选择
|selStochasticUniversalSampling()|	随机抽样选择
|selLexicase()|	词典选择，参考[这篇文章](http://faculty.hampshire.edu/lspector/pubs/lexicase-IEEE-TEC.pdf)
|selEpsilonLexicase()|	词典选择在连续值域上的扩展
|selAutomaticEpsilonLexicase()|	

### 2.常用选择操作介绍
#### 锦标赛选择：`deap.tools.selTournament(individuals, k, tournsize, fit_attr = 'fitness')`

锦标赛选择顾名思义，就是模拟锦标赛的方式，首先在族群中随机抽取`tournsize`个个体，然后从中选取具有最佳适应度的个体，将此过程重复k次，获得育种族群。`tournsize`越大，选择强度(selection intensity)越高，在选择之后留下的育种族群的平均适应度也就越高。比较常用的`tournsize`是2。

下图给出了由5个个体构成的族群中进行一次tournsize为3的锦标赛选择的过程。

![](https://upload-images.jianshu.io/upload_images/17867674-d94652a0f18353b2.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/1024/format/webp)

锦标赛选择相比于轮盘赌选择，通常能够有更快的收敛速度，在实际场景中应用较多。


#### 轮盘赌选择: `deap.tools.selRoulette(individuals, k, fit_attr = 'fitness')`
轮盘赌选择是最常见的选择策略，它可以看作是有放回的随机抽样。

在轮盘赌选择中，每个个体a_i被选中的概率P(a_i)与其适应度函数f(a_i)成正比：

$$P(a_i)=\frac{f(a_i)}{\sum_if(a_i)}$$

下图给出了与前文同样例子的轮盘赌选择：

![](https://upload-images.jianshu.io/upload_images/17867674-3c503fc666584cf5.jpg?imageMogr2/auto-orient/strip|imageView2/2/format/webp)


注意在适应度可能为负数时，不适合用轮盘赌选择。

在实际应用中，很多文章都指出轮盘赌选择的性能较差，在通常情况下都不如随机抽样选择和锦标赛选择。


#### 随机普遍抽样选择：`deap.tools.selStochasticUniversalSampling(individuals, k, fit_attr = 'fitness')`
随机普遍抽样选择是一种有多个指针的轮盘赌选择，其优点是能够保存族群多样性，而不会像轮盘赌一样，有较大几率对重复选择最优个体。

在与前文相同的例子中使用随机普遍抽样选择，设定指针数k为3，那么指针间距即为，如下图所示：

![](https://upload-images.jianshu.io/upload_images/17867674-c20b83385c700292.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/555/format/webp)

#### NSGA-II 选择：`deap.tools.selNSGA2(individuals, k, nd = 'standard')`

NSGA-II全称为 Nondominated sorting genetic algorithm II，是Kalyanmoy Deb于2002年提出的。该方法解决了前代NSGA的三个痛点：计算复杂度高；缺少精英选择；需要给定额外参数值。

在使用该函数时，需要注意族群中个体数量必须要比k值大，因为在该算法中，每个个体在返回的选择列表中至多出现一次。

以下是关于NSGA-II的资料
+ [NSGA-II论文](https://www.iitk.ac.in/kangal/Deb_NSGA-II.pdf)
+ [算法的C语言实现](https://www.iitk.ac.in/kangal/codes.shtml)

此外，一些基于排序的选择算法，如Linear ranking selection, Exponential ranking selection等，在DEAP中都没有给出直接的函数，需要自己实现。

### 3.选择操作代码示例

In [ ]:
# 定义问题
creator.create('FitnessMin', base.Fitness, weights=(-1.0,)) #优化目标：单变量，求最小值
creator.create('Individual', list, fitness = creator.FitnessMin) #创建Individual类，继承list

# 生成个体
IND_SIZE = 5
toolbox = base.Toolbox()
toolbox.register('Attr_float', np.random.rand)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Attr_float, n=IND_SIZE)

# 生成初始族群
N_POP = 10
toolbox.register('Population', tools.initRepeat, list, toolbox.Individual)
pop = toolbox.Population(n = N_POP)

# 定义评价函数
def evaluate(individual):
  return sum(individual), #注意这个逗号，即使是单变量优化问题，也需要返回tuple

# 评价初始族群
toolbox.register('Evaluate', evaluate)
fitnesses = map(toolbox.Evaluate, pop)
for ind, fit in zip(pop, fitnesses):
  ind.fitness.values = fit

# 选择方式1：锦标赛选择
toolbox.register('TourSel', tools.selTournament, tournsize = 2) # 注册Tournsize为2的锦标赛选择
selectedTour = toolbox.TourSel(pop, 5) # 选择5个个体
print('锦标赛选择结果：')
for ind in selectedTour:
  print(ind)
  print(ind.fitness.values)

# 选择方式2: 轮盘赌选择
toolbox.register('RoulSel', tools.selRoulette)
selectedRoul = toolbox.RoulSel(pop, 5)
print('轮盘赌选择结果：')
for ind in selectedRoul:
  print(ind)
  print(ind.fitness.values)

# 选择方式3: 随机普遍抽样选择
toolbox.register('StoSel', tools.selStochasticUniversalSampling)
selectedSto = toolbox.StoSel(pop, 5)
print('随机普遍抽样选择结果：')
for ind in selectedSto:
  print(ind)
  print(ind.fitness.values)

## 变异
变异过程就是从父代的基因出发，进行操作，最终得到子代基因的过程。通常包括交叉(Crossover)和突变(Mutation)两种操作。

### 1.DEAP内置的交叉(Crossover)操作
|函数|	简介|	适用编码方式
| --- | --- | ---
| cxOnePoint()|	单点交叉 |	实数、二进制
| cxTwoPoint()|	两点交叉 |	实数、二进制
| cxUniform()|	均匀交叉 |	实数、二进制
| cxPartialyMatched()|	部分匹配交叉PMX |	序列
| cxUniformPartialyMatched()|	PMX变种，改两点为均匀交叉 |	序列
| cxOrdered()|	有序交叉 |	序列
| cxBlend()|	混合交叉 |	实数
| cxESBlend()|	带进化策略的混合交叉 |	
| cxESTwoPoint()|	带进化策略的两点交叉 |	
| cxSimulatedBinary()|	模拟二值交叉 |	实数
| cxSimulatedBinaryBounded()|	有界模拟二值交叉 |	实数
| cxMessyOnePoint()|	混乱单点交叉 |	实数、二进制

### 2.常用交叉操作介绍
#### 单点交叉：`deap.tools.cxOnePoint(ind1, ind2)`
最简单的交叉方式，选择一个切口，将两条基因切开之后，交换尾部基因段。尽管该方法非常简单，但是多篇文章指出，该算法在各种实验中性能都被其他交叉算法吊打，因此算是一种不建议使用的loser algorithm。

![](https://upload-images.jianshu.io/upload_images/17867674-a761ddd2b66187eb.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/884/format/webp)

#### 两点交叉：`deap.tools.cxTwoPoint(ind1, ind2)`
用两个点切开基因之后，交换切出来的基因段。

![](https://upload-images.jianshu.io/upload_images/17867674-b08f2a38bd9540c7.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/857/format/webp)

#### 均匀交叉：`deap.tools.cxUniform(ind1, ind2, indpb)`
指定一个变异几率，两父代中的每个基因都以该几率交叉。

![](https://upload-images.jianshu.io/upload_images/17867674-61e22658b6147854.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/837/format/webp)

#### 部分匹配交叉PMX：`deap.tools.cxPartialyMatched(ind1, ind2)`
部分匹配交叉主要用于序列编码的个体，进行部分匹配交叉包括3个步骤：首先，选择父辈1的一段基因，复制到子代中；其次，查找父辈2中同位置的基因段，选择没有被复制的基因，建立一个映射关系；最后，进行冲突检查，如果基因有冲突，则通过建立的映射变换为无冲突的基因，保证形成的一对子代基因无冲突。

> 这里原作者讲的不是很清楚，如果有需要可以阅读PMX的[详细介绍](https://www.rubicite.com/Tutorials/GeneticAlgorithms/CrossoverOperators/PMXCrossoverOperator.aspx)

![](https://upload-images.jianshu.io/upload_images/17867674-b9913920c79990e8.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/783/format/webp)

当解决路径规划问题时，如果最优sub-subrouine越长，PMX交叉后就越难在子代中保留。

#### 有序交叉：`deap.tools.cxOrdered(ind1, ind2)`

![](https://upload-images.jianshu.io/upload_images/17867674-edcbb9ef2d1f30ce.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/753/format/webp)

#### 混合交叉BLX：`deap.tools.cxBlend(ind1, ind2, alpha)`
混合交叉由Eshelman和Schaffer在1993年提出，常见的混合交叉算子有$BLX-\alpha$与$BLX-\alpha\beta$两种，DEAP中内置的是前者。其具体算法如下：

+ 选择两个父代$X^{(t)}$和$Y^{(t)}$
+ 对基因i，计算$d_i=|x_i^{(t)} - y_i^{(t)}|$
+ 在区间$[min(x_i^{(t)},y_i^{(t)})-\alpha d_i, max(x_i^{(t)},y_i^{(t)})+\alpha d_i]$之间取随机数$u_i$
+ 将该随机数作为子代的片段，即$x_i^{(t+1)}=u_i$

对于任意$\alpha>0$，混合交叉会扩张搜索空间，因此应用于受到约束的变量时需要注意。有些研究认为$\alpha=0.5$时，搜索效果优于其他值。

#### 模拟二值交叉SBX：`deap.tools.cxSimulatedBinary(ind1, ind2, eta)`
SBX是在1995年由Deb和Agrawal提出来的。二进制编码有只能进行离散搜索，Hamming cliff等问题，而实数编码尽管能在连续域上操作，但是搜索能力较弱（此处搜索能力定义为给定一对父辈，产生任意子代的几率，可以用扩散系数表征）。模拟二值交叉试图综合二者的优势，在实数编码上模拟二进制编码的搜索特点。

参数$\eta_c$越大，产生的子代与父代越接近；该参数越小，产生的子代越可能与父代差距较大。

作者认为SBX在较难的测试中，表现比BLX-0.5要更优，尤其在多局部最优问题中表现出色。[论文原文](https://www.semanticscholar.org/paper/Simulated-Binary-Crossover-for-Continuous-Search-Deb-Agrawal/b8ee6b68520ae0291075cb1408046a7dff9dd9ad?p2df)

![](https://upload-images.jianshu.io/upload_images/17867674-3499dedb418d93ad.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/1024/format/webp)

#### 混乱单点交叉：`deap.tools.cxMessyOnePoint(ind1, ind2)`
作者质疑为何遗传算法中的基因都是如此有序：长短一致，编码方式整整齐齐，反而在自然界中这样的规律并不多见。因而作者提出了Messy GA，在这篇文章中，他将交叉操作拆分为cut与splice。Messy Crossover与一般的单点交叉最大的不同在于序列长度不会保持，如下图所示：

![](https://upload-images.jianshu.io/upload_images/17867674-d37372aca8c22311.jpg?imageMogr2/auto-orient/strip|imageView2/2/w/1024/format/webp)

### 3.交叉操作代码示例

In [ ]:
"""
官方提示最好不要直接用父代进行交叉，因为有些交叉算法是in-place运算的，
因此最好先复制，再进行交叉。
"""
# 创建两个序列编码个体
random.seed(42) # 保证结果可复现
IND_SIZE = 8
creator.create('FitnessMin', base.Fitness, weights=(-1.0, ))
creator.create('Individual', list, fitness = creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('Indices', random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register('Individual', tools.initIterate, creator.Individual, toolbox.Indices)

ind1, ind2 = [toolbox.Individual() for _ in range(2)]
print(ind1, '\n', ind2)

In [ ]:
# 单点交叉
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxOnePoint(child1, child2)
print(child1, '\n', child2)

In [ ]:
# 两点交叉
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxTwoPoint(child1, child2)
print(child1, '\n', child2)

In [ ]:
# 均匀交叉
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxUniform(child1, child2, 0.5)
print(child1, '\n', child2)

In [ ]:
# 部分匹配交叉
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxPartialyMatched(child1, child2)
print(child1, '\n', child2)
# 可以看到与之前交叉算子的明显不同，这里的每个序列都没有冲突

In [ ]:
# 有序交叉
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxOrdered(child1, child2)
print(child1, '\n', child2)

In [ ]:
# 混乱单点交叉
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxMessyOnePoint(child1, child2)
print(child1, '\n', child2)

### 4.DEAP内置的突变(Mutation)操作
|函数|	简介|	适用编码方式
|---|---|---
| mutGaussian()|	高斯突变|	实数
| mutShuffleIndexes()|	乱序突变|	实数、二进制、序列
| mutFlipBit()|	位翻转突变|	二进制
| mutPolynomialBounded()|	有界多项式突变|	实数
| mutUniformInt()|	均匀整数突变|	实数、序列
| mutESLogNormal()|		|

### 5.常用突变操作介绍
#### 高斯突变：`tools.mutGaussian(individual, mu, sigma, indpb)`
对个体序列中的每一个基因按概率变异，变异后的值为按均值为$\mu$，方差为$\sigma$的高斯分布选取的一个随机数。如果不希望均值发生变化，则应该将$\mu$设为0。
> 新的值为老值加上随机数

#### 乱序突变：`tools.mutShuffleIndexes(individual, indpb)`
将个体序列打乱顺序，每个基因位置变动的几率由indpb给出。

#### 位翻转突变：`tools.mutFlipBit(individual, indpb)`
对个体中的每一个基因按给定对变异概率取非。

#### 有界多项式突变：`tools.mutPolynomialBounded(individual, eta, low, up, indpb)`

多项式突变一般在多目标优化的**NSGA-II**算法中配合使用。其具体算法如下：

若$x_i\in[x^{(L)},x^{(U)}]$，突变后的个体$\overline{x_{i}}^{(1, t)}$由下式计算可得：

$$\overline{x_{i}}^{(1, t)}=x_{i}^{(1, t)}+\left(x^{(U)}-x^{(L)}\right) \overline{\delta}_{i}$$

其中参数$\overline{\delta}_{i}$服从多项式分布：

$$\mathcal{P}(\delta)=0.5\left(\eta_{\mathrm{m}}+1\right)(1-|\delta|)^{\eta_{\mathrm{m}}}$$

在具体计算时，首先在$[0,1]$中以均匀分布取一个随机数$u_i$再按下式计算$\overline{\delta}_{i}$：

$$\overline{\delta}_{i}=\left\{\begin{array}{ll}{\left(2 u_{i}\right)^{1 /\left(\eta_{m}+1\right)}-1,} & {\text { if } u_{i}<0.5} \\ {1-\left[2\left(1-u_{i}\right)\right]^{1 /\left(\eta_{m}+1\right)},} & {\text { if } u_{i} \geq 0.5}\end{array}\right.$$

Deb教授建议的参数$\eta_m$取$[20,100]$之间的数字，当参数取的越小，那么突变后的结果离突变前越近，影响大约为$O(1/\eta_m)$级。

#### 均匀整数突变：`tools.mutUniformInt(individual, low, up, indpb)`
对序列中的每一位按概率变异，变异后的值为[low, up]中按均匀分布随机选取的一个整数。

### 6.突变操作代码示例

In [ ]:
# 创建一个实数编码个体
random.seed(42) # 保证结果可复现
IND_SIZE = 5
creator.create('FitnessMin', base.Fitness, weights=(-1.0, ))
creator.create('Individual', list, fitness = creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('Attr_float', random.random)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Attr_float, IND_SIZE)

ind1 = toolbox.Individual()
print(ind1)

In [ ]:
# 高斯突变
mutant = toolbox.clone(ind1)
tools.mutGaussian(mutant, 3, 0.1, 1)
print(mutant)
# 可以看到当均值给到3之后，变异形成的个体均值从0.5也增大到了3附近

In [ ]:
# 乱序突变
mutant = toolbox.clone(ind1)
tools.mutShuffleIndexes(mutant, 0.5)
print(mutant)

In [ ]:
# 有界多项式突变
mutant = toolbox.clone(ind1)
tools.mutPolynomialBounded(mutant, 20, 0, 1, 0.5)
print(mutant)

In [ ]:
# 均匀整数突变
mutant = toolbox.clone(ind1)
tools.mutUniformInt(mutant, 1, 5, 0.5)
print(mutant)
# 可以看到在第二个位置生成了整数3

## 环境选择
环境选择也就是重插入(Reinsertion)，在选择、交叉和突变之后，得到的育种后代族群规模与父代相比可能增加或减少。为保持族群规模，需要将育种后代插入到父代中，替换父代种群的一部分个体，或者丢弃一部分育种个体。

重插入分为全局重插入(Global reinsertion)和本地重插入(Local reinsertion)两种，后者只有在使用含有本地邻域的算法时使用。常见的全局重插入操作有以下四种：

+ 完全重插入(Pure reinsertion)：产生与父代个体数量相当的配种个体，直接用配种个体生成新一代族群。
+ 均匀重插入(Uniform reinsertion)：产生比父代个体少的配种个体，用配种个体随机均匀地替换父代个体。
+ 精英重插入(Elitist reinsertion)：产生比父代个体少的配种个体，选取配种后代中适应度最好的一些个体，插入父代中，取代适应度较低的父代个体。
+ 精英保留重插入(Fitness-based reinsertion)：产生比父代个体多的配种个体，选取其中适应度最大的配种个体形成新一代族群。

通常来说后两种方式由于精英保留的缘故，收敛速度较快，因此比较推荐。

DEAP中没有设定专门的`reinsertion`操作，可以用选择操作选择中的`selBest`, `selWorst`,`selRandom`来对育种族群和父代族群进行操作。
